In [ ]:
%load_ext rpy2.ipython
from datetime import datetime, timedelta
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np

pandas2ri.activate()

In [ ]:
%%R
library("depmixS4")

### Import dataset

In [ ]:
df = pd.read_csv('/directory/file_name.csv')

#### Bin the channels - categories from 1

In [ ]:
df['Outgoing call'] = df['Outgoing call'].apply(lambda x: 2 if x>0 else 1)
df['Incoming call'] = df['Incoming call'].apply(lambda x: 2 if x>0 else 1)
df['Social media App'] = df['Social media App'].apply(lambda x: 2 if x>0 else 1)
df['Communication App'] = df['Communication App'].apply(lambda x: 2 if x>0 else 1)
df['Phone Usage'] = df['Phone Usage'].apply(lambda x: 2 if x>0 else 1)

#### Missing data

In [ ]:
#  If has_data is 0, set all other columns to NaN
df.loc[df['has_data'] == 0, 'Outgoing call'] = np.nan
df.loc[df['has_data'] == 0, 'Incoming call'] = np.nan
df.loc[df['has_data'] == 0, 'Social media App'] = np.nan
df.loc[df['has_data'] == 0, 'Communication App'] = np.nan
df.loc[df['has_data'] == 0, 'Phone Usage'] = np.nan

### Group model training

In [ ]:
# For training/testing split:
df_training = df[df['Testing1_Training0']==0]
all_available_segments = df_training.drop_duplicates(subset=['ID'])
training_segments = all_available_segments[all_available_segments['Testing1_Training0']==0]
df_subset = df_training[['ID', 'Outgoing call', 'Incoming call', 'Social media App', 'Communication App', 'Phone Usage']]

In [ ]:
# Get vector containing lengths of time series for each training participant
IDs = list(df_subset['ID'].unique())
sequence_length = []
for ID in IDs:
    length = len(df_subset[df_subset['ID']==ID])
    sequence_length.append(length)

In [ ]:
%%R -i df_subset,sequence_length
# Train model
seed_values <- c(10,21,39,42,58,63,77,84,96,150)

for (seed_value in seed_values)
    {
    set.seed(seed_value)
    start_time <- Sys.time()
    colnames(df_subset) <- c('ID', 'Outgoing.call', 'Incoming.call', 'Social.media.App', 'Communication.App', 'Phone.Usage')
    n_states <- 2
    mod <- depmix(list(Outgoing.call ~ 1 
                       , Incoming.call ~ 1
                       , Social.media.App ~ 1
                       , Communication.App ~ 1
                       , Phone.Usage ~ 1
                      ), data = df_subset, nstates = n_states,
     family = list(multinomial(link="identity")
                   , multinomial(link="identity"), multinomial(link="identity")
                   , multinomial(link="identity"), multinomial(link="identity")
                  ),
                  ntimes=as.vector(sequence_length)
                 )

    fm <- fit(mod)
    end_time <- Sys.time()
    print(end_time - start_time)

    filename = paste('/directory/file_name_', as.character(n_states), 'states_seed-value', as.character(seed_value), '.rda', sep='')
    saveRDS(fm, file =filename)  
}


### Quantifications

In [ ]:
%%R 
fm <- readRDS('/directory/file_name.rda')
print(logLik(fm))
print(summary(fm))

### Predict hidden state sequence for training segments (participants)

In [ ]:
%%R
pst_viterbi <- posterior(fm,type="viterbi")
group_state_sequence <- pst_viterbi['state']

In [ ]:
# Save hidden state sequence
training_overall_state_sequence = r['group_state_sequence']
df_training_overall_state_sequence = pd.DataFrame(columns=['State'])
df_training_overall_state_sequence['State'] = list(training_overall_state_sequence)[0]
df_training_overall_state_sequence.to_csv('/directory/file_name.csv', index=False)

In [ ]:
# If loading hidden state sequence
df_training_overall_state_sequence = pd.read_csv('/directory/file_name.csv')
df_training_overall_state_sequence.rename(columns={"State": "state"}, inplace=True)

In [ ]:
%%R -i df_training_overall_state_sequence
# If loading hidden state sequence
group_state_sequence <- df_training_overall_state_sequence

In [ ]:
# For training/testing split:
df_training = df[(df['Testing1_Training0']==0)]

In [ ]:
# Get vector containing lengths of time series for each participant
IDs = list(df_training['ID'].unique())
sequence_length_training = []
for ID in IDs:
    length = len(df_training[df_training['ID']==ID])
    sequence_length_training.append(length)

#### Training segment (participant) dwell times

In [ ]:
%%R -i sequence_length_training
all_training_segment_dwell_times <- list()
start <- 1
for (s in sequence_length_training)
    {
    end <- start + s - 1
    segment_state_sequence <- group_state_sequence[start:end,1]    
    dwell_time_state1 <- 100 * length(which(segment_state_sequence==1))/length(segment_state_sequence)
    dwell_time_state2 <- 100 * length(which(segment_state_sequence==2))/length(segment_state_sequence)
    segment_dwell_times <- list(dwell_time_state1,dwell_time_state2)
    all_training_segment_dwell_times <- append(all_training_segment_dwell_times,segment_dwell_times) 
    start <- end + 1
}

In [ ]:
all_training_segment_dwell_times = r['all_training_segment_dwell_times']

In [ ]:
dwell_times_state1 = all_training_segment_dwell_times[0:len(all_training_segment_dwell_times):2]
dwell_times_state2 = all_training_segment_dwell_times[1:len(all_training_segment_dwell_times):2]
dwell_times_state1 = [dwell_times_state1[i][0] for i in range(len(dwell_times_state1))]
dwell_times_state2 = [dwell_times_state2[i][0] for i in range(len(dwell_times_state2))]

In [ ]:
df_dwell_times = pd.DataFrame(columns=['ID', 'SFS', 'LONELINESS', 'AGE', 'LABEL', 'DATASET', 'dwell_times_state1','dwell_times_state2', 'Testing1_Training0'])
df_training_ID_unique = df_training.drop_duplicates(subset=['ID'])
df_dwell_times['ID'] = df_training_ID_unique['ID']
df_dwell_times['SFS'] = df_training_ID_unique['SFS']
df_dwell_times['LONELINESS'] = df_training_ID_unique['LONELINESS']
df_dwell_times['AGE'] = df_training_ID_unique['AGE']
df_dwell_times['LABEL'] = df_training_ID_unique['LABEL']
df_dwell_times['DATASET'] = df_training_ID_unique['DATASET']
df_dwell_times['dwell_times_state1'] = dwell_times_state1
df_dwell_times['dwell_times_state2'] = dwell_times_state2
df_dwell_times['Testing1_Training0'] = 0
df_dwell_times.to_csv('/directory/file_name.csv', index=False)

### Predict hidden state sequence for testing segments (participants)

In [ ]:
df_testing = df[(df['Testing1_Training0']==1)]
df_testing = df_testing.loc[df_testing['has_data']==1]

In [ ]:
# Get vector containing lengths of time series for each participant
df_testing = df_testing[['ID', 'Outgoing call', 'Incoming call', 'Social media App', 'Communication App', 'Phone Usage']]
df_testing_dummy = pd.DataFrame({'ID': [0], 'Outgoing call': [np.nan], 'Incoming call': [np.nan], 'Social media App': [np.nan], 'Communication App': [np.nan], 'Phone Usage': [np.nan]}) 
df_testing = pd.concat([df_testing, df_testing_dummy], ignore_index = True)  # Have to add in a fake missing row or else get errors with emission probabilities in mod_testing 
sequence_length_testing = []
for ID in list(df_testing['ID'].unique()):
    length = len(df_testing[df_testing['ID']==ID])
    sequence_length_testing.append(length)

In [ ]:
%%R
initial_probabilities <- getpars(fm)[1:2]
transition_matrix <- getpars(fm)[3:6]
emission_probabilities <- getpars(fm)[7:36]

In [ ]:
%%R -i df_testing,sequence_length_testing
colnames(df_testing) <- c('ID', 'Outgoing.call', 'Incoming.call', 'Social.media.App', 'Communication.App', 'Phone.Usage')
n_states <- 2
mod_testing <- depmix(list(Outgoing.call ~ 1
                   , Incoming.call ~ 1
                   , Social.media.App ~ 1
                   , Communication.App ~ 1
                   , Phone.Usage ~ 1
                  ), data = df_testing, nstates = n_states,
 family = list(multinomial(link="identity")
               , multinomial(link="identity"), multinomial(link="identity")
               , multinomial(link="identity"), multinomial(link="identity")
              ),
              ntimes=as.vector(sequence_length_testing),
              respstart=emission_probabilities,
              trstart=transition_matrix,
              instart=initial_probabilities
             )

In [ ]:
%%R
pst_viterbi <- posterior(mod_testing,type="viterbi")
testing_overall_state_sequence <- pst_viterbi['state']

In [ ]:
# Save hidden state sequence
testing_overall_state_sequence = r['testing_overall_state_sequence']
testing_overall_state_sequence = testing_overall_state_sequence[:]
df_testing_overall_state_sequence = pd.DataFrame(columns=['State'])
df_testing_overall_state_sequence['State'] = list(testing_overall_state_sequence)[0][:-1]
df_testing_overall_state_sequence.to_csv('/directory/file_name.csv', index=False)

In [ ]:
%%R -i sequence_length_testing
all_testing_segment_dwell_times <- list()
start <- 1
for (s in sequence_length_testing)
    {
    end <- start + s - 1
    testing_state_sequence <- testing_overall_state_sequence[start:end,1]    
    dwell_time_state1 <- 100 * length(which(testing_state_sequence==1))/length(testing_state_sequence)
    dwell_time_state2 <- 100 * length(which(testing_state_sequence==2))/length(testing_state_sequence)
    segment_dwell_times <- list(dwell_time_state1,dwell_time_state2)
    all_testing_segment_dwell_times <- append(all_testing_segment_dwell_times,segment_dwell_times) 
    start <- end + 1
}

In [ ]:
all_testing_segment_dwell_times = r['all_testing_segment_dwell_times']  # last 2 elements are from the dummy missing row

In [ ]:
dwell_times_state1 = all_testing_segment_dwell_times[0:len(all_testing_segment_dwell_times):2]
dwell_times_state2 = all_testing_segment_dwell_times[1:len(all_testing_segment_dwell_times):2]
dwell_times_state1 = [dwell_times_state1[i][0] for i in range(len(dwell_times_state1)-1)]  # don't need the last row as this is the dummy row
dwell_times_state2 = [dwell_times_state2[i][0] for i in range(len(dwell_times_state2)-1)]

In [ ]:
df_testing = df[(df['Testing1_Training0']==1)]

In [ ]:
df_dwell_times = pd.DataFrame(columns=['ID', 'SFS', 'LONELINESS', 'LABEL', 'AGE', 'SEX', 'RACE',
       'EDUCATION YEARS', 'COUNTRY', 'DATASET', 'dwell_times_state1', 'dwell_times_state2', 'Testing1_Training0'])
df_testing_ID_unique = df_testing.drop_duplicates(subset=['ID'])
df_dwell_times['ID'] = df_testing_ID_unique['ID']
df_dwell_times['SFS'] = df_testing_ID_unique['SFS']
df_dwell_times['LONELINESS'] = df_testing_ID_unique['LONELINESS']
df_dwell_times['LABEL'] = df_testing_ID_unique['LABEL']
df_dwell_times['AGE'] = df_testing_ID_unique['AGE']
df_dwell_times['SEX'] = df_testing_ID_unique['SEX']
df_dwell_times['RACE'] = df_testing_ID_unique['RACE']
df_dwell_times['EDUCATION YEARS'] = df_testing_ID_unique['EDUCATION YEARS']
df_dwell_times['COUNTRY'] = df_testing_ID_unique['COUNTRY']
df_dwell_times['DATASET'] = df_testing_ID_unique['DATASET']
df_dwell_times['dwell_times_state1'] = dwell_times_state1
df_dwell_times['dwell_times_state2'] = dwell_times_state2
df_dwell_times['Testing1_Training0'] = 1
df_dwell_times.to_csv('/directory/file_name.csv', index=False)